# Script for a period

In [11]:
import requests
import json
import pandas as pd
from datetime import date
from clickhouse_connect import get_client
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os
import time


load_dotenv()

True

In [12]:
KeyGuten = os.getenv('KeyGuten')
KeyGiper = os.getenv('KeyGiper')
KeyKitchen = os.getenv('KeyKitchen')
KeySmart = os.getenv("KeySmart")

## Списки кампаний

Link: https://dev.wildberries.ru/openapi/promotion#tag/Kampanii/paths/~1adv~1v1~1promotion~1count/get

In [13]:


# API endpoint
url = 'https://advert-api.wildberries.ru/adv/v1/promotion/count'

# Headers including the API key for authentication
headers_guten = {
    'Authorization': KeyGuten,
    'Accept': 'application/json'
}

headers_giper = {
    'Authorization': KeyGiper,
    'Accept': 'application/json'
}

headers_kitchen = {
    'Authorization': KeyKitchen,
    'Accept': 'application/json'
}

headers_smart = {
    'Authorization': KeySmart,
    'Accept': 'application/json'
}

# Make the GET request
response_guten = requests.get(url, headers=headers_guten)
response_giper = requests.get(url, headers=headers_giper)
response_kitchen = requests.get(url, headers=headers_kitchen)
response_smart = requests.get(url, headers=headers_smart)

# Check if the request was successful
if response_guten.status_code == 200 and response_giper.status_code == 200 and response_kitchen.status_code == 200 and response_smart.status_code == 200:
    # Parse the JSON response
    data_guten = response_guten.json()
    data_giper = response_giper.json()
    data_kitchen = response_kitchen.json()
    data_smart = response_smart.json()
    print("Data retrieved successfully")
else:
    print(f"Failed to retrieve data. Status code: {response_guten.status_code,response_giper.status_code,response_giper.status_code,response_smart.status_code }")
    print(f"Response: {response_guten.text, response_giper.text,response_giper.text,response_smart.text }")

Data retrieved successfully


In [14]:
data_giper

{'adverts': [{'type': 9,
   'status': 11,
   'count': 3,
   'advert_list': [{'advertId': 22950974,
     'changeTime': '2025-03-03T12:35:20.701855+03:00'},
    {'advertId': 22951078, 'changeTime': '2025-03-03T12:47:32.279379+03:00'},
    {'advertId': 22951454, 'changeTime': '2025-04-14T06:59:11.104603+03:00'}]},
  {'type': 9,
   'status': 9,
   'count': 8,
   'advert_list': [{'advertId': 22844893,
     'changeTime': '2025-04-16T13:29:27.258157+03:00'},
    {'advertId': 22950405, 'changeTime': '2025-04-08T12:12:36.523812+03:00'},
    {'advertId': 22950831, 'changeTime': '2025-04-08T12:08:00.446877+03:00'},
    {'advertId': 22951156, 'changeTime': '2025-04-03T14:59:05.072079+03:00'},
    {'advertId': 22951268, 'changeTime': '2025-04-03T12:34:55.94527+03:00'},
    {'advertId': 22954069, 'changeTime': '2025-04-03T12:36:30.266965+03:00'},
    {'advertId': 25054241, 'changeTime': '2025-04-24T11:56:40.875874+03:00'},
    {'advertId': 25055099, 'changeTime': '2025-04-24T12:14:49.036817+03:00'}]

In [15]:
# Flatten the JSON data
df_guten = pd.json_normalize(
    data_guten['adverts'],
    record_path='advert_list', 
    meta=['type', 'status', 'count']
)
df_giper = pd.json_normalize(
    data_giper['adverts'],
    record_path='advert_list', 
    meta=['type', 'status', 'count']
)
df_kitchen = pd.json_normalize(
    data_kitchen['adverts'],
    record_path='advert_list', 
    meta=['type', 'status', 'count']
)
df_smart = pd.json_normalize(
    data_smart['adverts'],
    record_path='advert_list', 
    meta=['type', 'status', 'count']
)

# Convert 'changeTime' to datetime format
df_guten['changeTime'] = pd.to_datetime(df_guten['changeTime'])
df_giper['changeTime'] = pd.to_datetime(df_giper['changeTime'])
df_kitchen['changeTime'] = pd.to_datetime(df_kitchen['changeTime'])
df_smart['changeTime'] = pd.to_datetime(df_smart['changeTime'])

# Filter rows where the year of 'changeTime' is 2024 or later
#df_guten = df_guten[df_guten['changeTime'].dt.year >= 2024]
#df_giper = df_giper[df_giper['changeTime'].dt.year >= 2024]
#df_kitchen = df_kitchen[df_kitchen['changeTime'].dt.year >= 2024]
#df_smart = df_smart[df_smart['changeTime'].dt.year >= 2024]

# Reset the index and drop the old index
df_guten = df_guten.reset_index(drop=True)
df_giper = df_giper.reset_index(drop=True)
df_kitchen = df_kitchen.reset_index(drop=True)
df_smart = df_smart.reset_index(drop=True)

# Display the updated DataFrame
#df_guten
#df_giper
#df_kitchen

In [16]:
#unique_change_times = df_guten['advertId'].nunique()
#print(unique_change_times)

### Информация о кампаниях


Link: https://dev.wildberries.ru/openapi/promotion#tag/Kampanii/paths/~1adv~1v1~1promotion~1adverts/post

In [17]:
# Create chunks of 50 campaign IDs each
chunk_size = 50
campaign_chunks_guten = [df_guten['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df_guten), chunk_size)]
campaign_chunks_giper = [df_giper['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df_giper), chunk_size)]
campaign_chunks_kitchen = [df_kitchen['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df_kitchen), chunk_size)]
campaign_chunks_smart = [df_smart['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df_smart), chunk_size)]

# Print the list of all chunks
print("Campaign Chunks for WB-GutenTech:", campaign_chunks_guten)
print("Campaign Chunks for WB-ГиперМаркет:", campaign_chunks_giper)
print("Campaign Chunks for WB-KitchenAid:", campaign_chunks_kitchen)
print("Campaign Chunks for WB-Smart-Market:", campaign_chunks_smart)

# Define the API endpoint
url = "https://advert-api.wildberries.ru/adv/v1/promotion/adverts"

# Define the query parameters
query_params = {
    "order": "create",  # Order by the "change" field
    "direction":"desc"
}


# Function to fetch campaign data
def fetch_campaign_data(chunks, headers, project_name):
    all_campaign_data = []
    for idx, chunk in enumerate(chunks):
        response = requests.post(url, params=query_params, json=chunk, headers=headers)
        if response.status_code == 200:
            data = response.json()
            all_campaign_data.extend(data)
            print(f"Data retrieved successfully for {project_name} chunk {idx + 1}")
        else:
            print(f"Error for {project_name} chunk {idx + 1}: {response.status_code}, {response.text}")
        time.sleep(1)  # Add a delay to avoid hitting API rate limits
    return all_campaign_data

# Fetch campaign data for each project
all_campaign_data_guten = fetch_campaign_data(campaign_chunks_guten, headers_guten, 'WB-GutenTech')
all_campaign_data_giper = fetch_campaign_data(campaign_chunks_giper, headers_giper, 'WB-ГиперМаркет')
all_campaign_data_kitchen = fetch_campaign_data(campaign_chunks_kitchen, headers_kitchen, 'WB-KitchenAid')
all_campaign_data_smart = fetch_campaign_data(campaign_chunks_smart, headers_smart, 'WB-Smart-Market')

# Create DataFrames from the campaign data
campaigns_guten = pd.DataFrame(all_campaign_data_guten)
campaigns_giper = pd.DataFrame(all_campaign_data_giper)
campaigns_kitchen = pd.DataFrame(all_campaign_data_kitchen)
campaigns_smart = pd.DataFrame(all_campaign_data_smart)

# Sort each DataFrame by 'createTime' in descending order
campaigns_guten = campaigns_guten.sort_values(by='createTime', ascending=False)
campaigns_giper = campaigns_giper.sort_values(by='createTime', ascending=False)
campaigns_kitchen = campaigns_kitchen.sort_values(by='createTime', ascending=False)
campaigns_smart = campaigns_smart.sort_values(by='createTime', ascending=False)

campaigns_guten['Project'] = 'WB-GutenTech'
campaigns_giper['Project'] = 'WB-ГиперМаркет'
campaigns_kitchen['Project'] = 'WB-KitchenAid'
campaigns_smart['Project'] = 'WB-Smart-Market'

# Concatenate the DataFrames
combined_campaigns = pd.concat([campaigns_guten, campaigns_giper, campaigns_kitchen,campaigns_smart], ignore_index=True)

combined_campaigns['Marketplace'] = 'Wildberries'

print("Combined Campaign Data")
#campaigns_kitchen

Campaign Chunks for WB-GutenTech: [[16530753, 18447862, 18870595, 18870619, 18870853, 19297608, 19297685, 20124734, 20139198, 20487381, 21202453, 21203996, 21212396, 21497699, 21497844, 21498159, 21498270, 21554091, 21554118, 21577713, 21624823, 21821064, 21821164, 21821199, 21883272, 21883285, 21965489, 22255122, 22846516, 22846531, 23230882, 23408417, 24780937, 7202847, 213920, 213946, 213954, 225156, 232543, 232547, 238682, 244654, 244667, 354788, 354832, 354853, 535702, 535721, 535733, 535812], [535864, 535918, 535940, 535963, 535994, 536002, 536018, 536060, 536362, 536369, 536385, 536415, 536427, 536485, 536514, 536537, 536549, 536568, 536610, 536617, 536631, 536646, 536659, 788466, 788472, 788580, 788586, 788636, 788641, 789178, 789191, 953420, 953764, 953774, 958835, 960520, 960525, 960529, 960544, 960551, 960611, 960618, 960621, 960624, 960626, 960675, 960682, 960689, 960693, 960701], [1005700, 1320657, 1320668, 1320684, 2594125, 2594226, 2594682, 2594752, 2595105, 2595113, 285

In [18]:
# Keep only the desired columns
columns_to_keep = ["endTime", "createTime", "startTime","name", "advertId", "status", "type","Project","Marketplace"]
# Ensure filtered_df is a copy of the slice, not a view
filtered_df = combined_campaigns[columns_to_keep].copy()
filtered_df['endTime'] = pd.to_datetime(filtered_df['endTime'], format='mixed').dt.date
filtered_df['createTime'] = pd.to_datetime(filtered_df['createTime'], format='mixed').dt.date
filtered_df['startTime'] = pd.to_datetime(filtered_df['startTime'], format='mixed').dt.date

# Mapping dictionaries for 'status' and 'type'
status_mapping = {
    -1: "Кампания в процессе удаления",
    4: "Готова к запуску",
    7: "Кампания завершена",
    8: "Отказался",
    9: "Идут показы",
    11: "Кампания на паузе"
}

type_mapping = {
    4: "Кампания в каталоге (устаревший тип)",
    5: "Кампания в карточке товара (устаревший тип)",
    6: "Кампания в поиске (устаревший тип)",
    7: "Кампания в рекомендациях на главной странице (устаревший тип)",
    8: "Автоматическая кампания",
    9: "Аукцион"
}

# Replace numeric values with their string descriptions
filtered_df['status'] = filtered_df['status'].replace(status_mapping)
filtered_df['type'] = filtered_df['type'].replace(type_mapping)

# Display the updated DataFrame
filtered_df

,endTime,createTime,startTime,name,advertId,status,type,Project,Marketplace
0,2100-01-01,2025-04-23,2025-04-23,Patriot_Опрыскиватель 12АС_23.04.2025,25031430,Идут показы,Аукцион,WB-GutenTech,Wildberries
1,2100-01-01,2025-04-23,2025-04-23,Oclean_Бюджетная щетка_23.04.2025,25017505,Идут показы,Аукцион,WB-GutenTech,Wildberries
2,2100-01-01,2025-04-21,2025-04-21,AND_Тонометр 888Е_21.04.2025,24960374,Идут показы,Аукцион,WB-GutenTech,Wildberries
3,2100-01-01,2025-04-14,2025-04-22,AND_Тонометр 777AC_14.04.2025,24780966,Кампания на паузе,Автоматическая кампания,WB-GutenTech,Wildberries
4,2100-01-01,2025-04-14,2025-04-14,AND_Тонометр 1300_14.04.2025,24780937,Идут показы,Автоматическая кампания,WB-GutenTech,Wildberries
...,...,...,...,...,...,...,...,...,...
481,2024-06-23,2024-03-21,2024-04-18,Насадки Oral-B Автореклама,15562382,Кампания завершена,Автоматическая кампания,WB-Smart-Market,Wildberries
482,2100-01-01,2024-03-21,2025-02-20,Braun_Электробритвы_22.10.2024,15562363,Идут показы,Автоматическая кампания,WB-Smart-Market,Wildberries
483,2024-10-11,2024-03-21,2024-07-02,Электрические зубные щетки Oral-B Поиск + Каталог,15562321,Кампания завершена,Аукцион,WB-Smart-Market,Wildberries
484,2024-10-15,2024-03-21,2024-08-27,Эпиляторы Braun_7 SKU_27.08.2024,15562282,Кампания завершена,Автоматическая кампания,WB-Smart-Market,Wildberries


## Статистика кампаний


Link: https://dev.wildberries.ru/openapi/analytics#tag/Statistika-po-prodvizheniyu/paths/~1adv~1v2~1fullstats/post

Данные вернутся для кампаний в статусах:

* 7 — завершена
* 9 — идут показы
* 11 — пауза из-за расхода бюджета

Response Schema: 

* dates	/ Array of strings <date> [ items <date > ]/ Даты, за которые необходимо выдать информацию.
* views	/ integer/ Количество просмотров./ За все дни, по всем артикулам WB и платформам.
* clicks	/ integer/ Количество кликов./ За все дни, по всем артикулам WB и платформам.
* ctr	/ number/ Показатель кликабельности./ Отношение числа кликов к количеству показов. Выражается в процентах./ За все дни, по всем артикулам WB и платформам.
* cpc	/ number/ Средняя стоимость клика, ₽./ За все дни, по всем артикулам WB и платформам.
* sum	/ number/ Затраты, ₽./ За все дни, по всем артикулам WB и платформам.
* atbs	/ integer/ Количество добавлений товаров в корзину./ За все дни, по всем артикулам WB и платформам.
* orders	/ integer/ Количество заказов./ За все дни, по всем артикулам WB и платформам.
* cr	/ integer/ CR(conversion rate) — это отношение количества заказов к общему количеству посещений кампании./ За все дни, по всем артикулам WB и платформам.
* shks	/ integer/ Количество заказанных товаров, шт./ За все дни, по всем артикулам WB и платформам.
* sum_price	/ number/ Заказов на сумму, ₽/ За все дни, по всем артикулам WB и платформам.
* days	/ Array of objects (Days)/ Статистка по дням
    Array 
    * date	/ string <date-time>/ Дата, за которую представлены данные/ 
    * views	/ integer/ Количество просмотров/ 
    * clicks	/ integer/ Количество кликов/ 
    * ctr	/ number/ Показатель кликабельности, отношение числа кликов к количеству показов, %/ 
    * cpc	/ number/ Средняя стоимость клика, ₽./ 
    * sum	/ number/ Затраты, ₽./ 
    * atbs	/ integer/ Количество добавлений товаров в корзину/ 
    * orders	/ integer/ Количество заказов/ 
    * cr	/ integer/ CR(conversion rate) — отношение количества заказов к общему количеству посещений кампании/ 
    * shks	/ integer/ Количество заказанных товаров, шт./ 
    * sum_price	/ number/ Заказов на сумму, ₽/ 
    * apps	/ Array of objects/ Блок информации о платформе
    Array 
        - views	/ integer/ Количество просмотров/ 
        - clicks	/ integer/ Количество кликов/ 
        - ctr	/ number/ Показатель кликабельности, отношение числа кликов к количеству показов, %/ 
        - cpc	/ number/ Средняя стоимость клика, ₽./ 
        - sum	/ number/ Затраты, ₽./ 
        - atbs	/ integer/ Количество добавлений товаров в корзину/ 
        - orders	/ integer/ Количество заказов/ 
        - cr	/ integer/ CR(conversion rate) — это отношение количества заказов к общему количеству посещений кампании/ 
        - shks	/ integer/ Количество заказанных товаров, шт./ 
        - sum_price	/ number/ Заказов на сумму, ₽/ 
        - nm	/ Array of objects/ Блок статистики по артикулам WB/ 
        - appType	/ integer/ Тип платформы (1 - сайт, 32 - Android, 64 - IOS)
* boosterStats	/ Array of objects (BoosterStats)/ Статистика по средней позиции товара на страницах поисковой выдачи и каталога (для автоматических кампаний).
* advertId	/ integer/ ID кампании

In [19]:
# API endpoint
url = "https://advert-api.wildberries.ru/adv/v2/fullstats"

# Define the date interval
date_interval = {
    "begin": "2025-04-07",  # Replace with your start date
    "end": "2025-04-07"     # Replace with your end date
}

# Create chunks of 100 campaign IDs each
chunk_size = 100

### Guten

In [20]:
# Assume campaigns_guten DataFrame is already defined and contains 'advertId'
# Create chunks of 100 campaign IDs each
campaign_chunks = [campaigns_guten['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaigns_guten), chunk_size)]

# List to store all campaign data
all_campaign_data_guten = []

# Function to fetch campaign data
def fetch_campaign_data(chunks, headers, date_interval):
    all_data = []
    for idx, chunk in enumerate(chunks):
        # Construct the payload for the current chunk
        payload = [{"id": campaign_id, "interval": date_interval} for campaign_id in chunk]

        # Send the POST request
        response = requests.post(url, headers=headers, json=payload)

        # Add a delay to avoid hitting API rate limits
        time.sleep(65)

        # Check the response status
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            all_data.extend(data)
            print(f"Data retrieved successfully for chunk {idx + 1}")
        else:
            print(f"Error for chunk {idx + 1}: {response.status_code}, {response.text}")
    return all_data

# Fetch campaign data
all_campaign_data_guten = fetch_campaign_data(campaign_chunks, headers_guten, date_interval)
# Combine all campaign data into a DataFrame
campaign_guten_df = pd.json_normalize(all_campaign_data_guten)

# Save the combined data to a text file
with open("response_data_guten.txt", "w", encoding="utf-8") as file:
    json.dump(all_campaign_data_guten, file, indent=4, ensure_ascii=False)

# Display the combined DataFrame
print("Combined Campaign Data")

Data retrieved successfully for chunk 1
Data retrieved successfully for chunk 2
Error for chunk 3: 400, {"error":"Invalid Params: there are no companies with correct intervals"}
Error for chunk 4: 400, {"error":"Invalid Params: there are no companies with correct intervals"}
Error for chunk 5: 400, {"error":"Invalid Params: there are no companies with correct intervals"}
Combined Campaign Data


### Giper

In [21]:
campaign_chunks = [campaigns_giper['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaigns_giper), chunk_size)]

# List to store all campaign data
all_campaign_data_giper = []

# Function to fetch campaign data
def fetch_campaign_data(chunks, headers, date_interval):
    all_data = []
    for idx, chunk in enumerate(chunks):
        # Construct the payload for the current chunk
        payload = [{"id": campaign_id, "interval": date_interval} for campaign_id in chunk]

        # Send the POST request
        response = requests.post(url, headers=headers, json=payload)

        # Add a delay to avoid hitting API rate limits
        time.sleep(65)

        # Check the response status
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            all_data.extend(data)
            print(f"Data retrieved successfully for chunk {idx + 1}")
        else:
            print(f"Error for chunk {idx + 1}: {response.status_code}, {response.text}")
    return all_data
# Fetch campaign data
all_campaign_data_giper = fetch_campaign_data(campaign_chunks, headers_giper, date_interval)

Data retrieved successfully for chunk 1


### Kitchen

In [22]:
chunk_size = 100
campaign_chunks = [campaigns_kitchen['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaigns_kitchen), chunk_size)]

# List to store all campaign data
all_campaign_data_kitchen = []

# Function to fetch campaign data
def fetch_campaign_data(chunks, headers, date_interval):
    all_data = []
    for idx, chunk in enumerate(chunks):
        # Construct the payload for the current chunk
        payload = [{"id": campaign_id, "interval": date_interval} for campaign_id in chunk]

        # Send the POST request
        response = requests.post(url, headers=headers, json=payload)

        # Add a delay to avoid hitting API rate limits
        time.sleep(65)

        # Check the response status
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            all_data.extend(data)
            print(f"Data retrieved successfully for chunk {idx + 1}")
        else:
            print(f"Error for chunk {idx + 1}: {response.status_code}, {response.text}")
    return all_data

# Fetch campaign data
all_campaign_data_kitchen = fetch_campaign_data(campaign_chunks, headers_kitchen, date_interval)


Data retrieved successfully for chunk 1


### Smart

In [23]:
campaign_chunks = [campaigns_smart['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaigns_smart), chunk_size)]

# List to store all campaign data
all_campaign_data_smart = []

# Function to fetch campaign data
def fetch_campaign_data(chunks, headers, date_interval):
    all_data = []
    for idx, chunk in enumerate(chunks):
        # Construct the payload for the current chunk
        payload = [{"id": campaign_id, "interval": date_interval} for campaign_id in chunk]

        # Send the POST request
        response = requests.post(url, headers=headers, json=payload)

        # Add a delay to avoid hitting API rate limits
        time.sleep(65)

        # Check the response status
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            all_data.extend(data)
            print(f"Data retrieved successfully for chunk {idx + 1}")
        else:
            print(f"Error for chunk {idx + 1}: {response.status_code}, {response.text}")
    return all_data

# Fetch campaign data
all_campaign_data_smart = fetch_campaign_data(campaign_chunks, headers_smart, date_interval)

Data retrieved successfully for chunk 1


In [24]:
campaign_guten_df

,views,clicks,ctr,cpc,sum,atbs,orders,cr,shks,sum_price,days,boosterStats,advertId,interval.begin,interval.end
0,5006,230,4.59,2.81,645.77,32,12,5.22,12,26407,"[{'date': '2025-04-07T03:00:00+03:00', 'views'...","[{'date': '2025-04-07T00:00:00Z', 'nm': 171562...",21554118,2025-04-07,2025-04-07
1,574,12,2.09,7.15,85.74,1,0,0.00,0,0,"[{'date': '2025-04-07T03:00:00+03:00', 'views'...",NaN,21203075,2025-04-07,2025-04-07
2,9910,505,5.10,3.69,1861.26,88,23,4.55,23,49609,"[{'date': '2025-04-07T03:00:00+03:00', 'views'...","[{'date': '2025-04-07T00:00:00Z', 'nm': 159121...",22846516,2025-04-07,2025-04-07
3,1681,71,4.22,15.39,1092.51,10,2,2.82,2,5846,"[{'date': '2025-04-07T03:00:00+03:00', 'views'...","[{'date': '2025-04-07T00:00:00Z', 'nm': 234921...",21883285,2025-04-07,2025-04-07
4,1673,33,1.97,8.78,289.70,4,0,0.00,0,0,"[{'date': '2025-04-07T03:00:00+03:00', 'views'...",NaN,21204224,2025-04-07,2025-04-07
5,953,26,2.73,7.44,193.47,0,0,0.00,0,0,"[{'date': '2025-04-07T03:00:00+03:00', 'views'...","[{'date': '2025-04-07T00:00:00Z', 'nm': 286307...",21577713,2025-04-07,2025-04-07
6,2744,94,3.43,17.81,1673.81,9,1,1.06,1,3140,"[{'date': '2025-04-07T03:00:00+03:00', 'views'...","[{'date': '2025-04-07T00:00:00Z', 'nm': 159121...",21883272,2025-04-07,2025-04-07
7,2842,85,2.99,6.85,582.63,8,1,1.18,1,6335,"[{'date': '2025-04-07T03:00:00+03:00', 'views'...",NaN,21202745,2025-04-07,2025-04-07
8,4337,172,3.97,5.01,861.05,23,8,4.65,8,19646,"[{'date': '2025-04-07T03:00:00+03:00', 'views'...","[{'date': '2025-04-07T00:00:00Z', 'nm': 159121...",22846531,2025-04-07,2025-04-07
9,1197,34,2.84,7.04,239.40,1,0,0.00,0,0,"[{'date': '2025-04-07T03:00:00+03:00', 'views'...",NaN,21203170,2025-04-07,2025-04-07


In [25]:
print(campaign_guten_df["advertId"].nunique())

#unique_change_times = df_guten['advertId'].nunique()
#print(unique_change_times)

43


In [26]:
# Flatten the JSON data
flattened_data_guten = []
flattened_data_giper = []
flattened_data_kitchen = []
flattened_data_smart = []


for entry in all_campaign_data_guten:
    advertId = entry["advertId"]  # Extract the advertId
    for day in entry["days"]:
        date = day["date"]
        for app in day["apps"]:
            for nm in app["nm"]:
                flattened_data_guten.append({
                    "date": date,
                    "nmId": nm["nmId"],
                    "name": nm["name"],
                    "views": nm["views"],
                    "clicks": nm["clicks"],
                    "ctr": nm["ctr"],
                    "cpc": nm["cpc"],
                    "sum": nm["sum"],
                    "atbs": nm["atbs"],
                    "orders": nm["orders"],
                    "cr": nm["cr"],
                    "shks": nm["shks"],
                    "sum_price": nm["sum_price"],
                    "advertId": advertId  # Add advertId to each row
                })
                
for entry in all_campaign_data_giper:
    advertId = entry["advertId"]  # Extract the advertId
    for day in entry["days"]:
        date = day["date"]
        for app in day["apps"]:
            for nm in app["nm"]:
                flattened_data_giper.append({
                    "date": date,
                    "nmId": nm["nmId"],
                    "name": nm["name"],
                    "views": nm["views"],
                    "clicks": nm["clicks"],
                    "ctr": nm["ctr"],
                    "cpc": nm["cpc"],
                    "sum": nm["sum"],
                    "atbs": nm["atbs"],
                    "orders": nm["orders"],
                    "cr": nm["cr"],
                    "shks": nm["shks"],
                    "sum_price": nm["sum_price"],
                    "advertId": advertId  # Add advertId to each row
                })

for entry in all_campaign_data_kitchen:
    advertId = entry["advertId"]  # Extract the advertId
    for day in entry["days"]:
        date = day["date"]
        for app in day["apps"]:
            for nm in app["nm"]:
                flattened_data_kitchen.append({
                    "date": date,
                    "nmId": nm["nmId"],
                    "name": nm["name"],
                    "views": nm["views"],
                    "clicks": nm["clicks"],
                    "ctr": nm["ctr"],
                    "cpc": nm["cpc"],
                    "sum": nm["sum"],
                    "atbs": nm["atbs"],
                    "orders": nm["orders"],
                    "cr": nm["cr"],
                    "shks": nm["shks"],
                    "sum_price": nm["sum_price"],
                    "advertId": advertId  # Add advertId to each row
                })
                
for entry in all_campaign_data_smart:
    advertId = entry["advertId"]  # Extract the advertId
    for day in entry["days"]:
        date = day["date"]
        for app in day["apps"]:
            for nm in app["nm"]:
                flattened_data_smart.append({
                    "date": date,
                    "nmId": nm["nmId"],
                    "name": nm["name"],
                    "views": nm["views"],
                    "clicks": nm["clicks"],
                    "ctr": nm["ctr"],
                    "cpc": nm["cpc"],
                    "sum": nm["sum"],
                    "atbs": nm["atbs"],
                    "orders": nm["orders"],
                    "cr": nm["cr"],
                    "shks": nm["shks"],
                    "sum_price": nm["sum_price"],
                    "advertId": advertId  # Add advertId to each row
                })
                

# Create a DataFrame
df_guten = pd.DataFrame(flattened_data_guten)
df_giper = pd.DataFrame(flattened_data_giper)
df_kitchen = pd.DataFrame(flattened_data_kitchen)
df_smart = pd.DataFrame(flattened_data_smart)
# Convert the 'date' column to datetime
df_guten["date"] = pd.to_datetime(df_guten["date"])
df_giper["date"] = pd.to_datetime(df_giper["date"])
df_kitchen["date"] = pd.to_datetime(df_kitchen["date"])
df_smart["date"] = pd.to_datetime(df_smart["date"])
# Remove timezone information (make it timezone-naive)
df_guten["date"] = df_guten["date"].dt.tz_localize(None)
df_giper["date"] = df_giper["date"].dt.tz_localize(None)
df_kitchen["date"] = df_kitchen["date"].dt.tz_localize(None)
df_smart["date"] = df_smart["date"].dt.tz_localize(None)

# Display the DataFrame
df_guten

,date,nmId,name,views,clicks,ctr,cpc,sum,atbs,orders,cr,shks,sum_price,advertId
0,2025-04-07 03:00:00,171562375,Ингалятор небулайзер компрессорный детский CN-...,144,4,2.78,4.72,18.89,0,0,0.00,0,0,21554118
1,2025-04-07 03:00:00,171562370,Ингалятор небулайзер компрессорный детский CN-...,259,6,2.32,5.44,32.64,0,0,0.00,0,0,21554118
2,2025-04-07 03:00:00,171562375,Ингалятор небулайзер компрессорный детский CN-...,1116,57,5.11,2.50,142.40,6,2,3.51,2,5318,21554118
3,2025-04-07 03:00:00,171562370,Ингалятор небулайзер компрессорный детский CN-...,2275,92,4.04,3.18,292.37,20,8,8.70,8,16394,21554118
4,2025-04-07 03:00:00,171562370,Ингалятор небулайзер компрессорный детский CN-...,752,38,5.05,2.59,98.34,1,1,2.63,1,2036,21554118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
639,2025-04-07 03:00:00,46292575,Яйцеварка электрическая автоматическая E 9,116,2,1.72,12.97,25.94,1,0,0.00,0,0,18870619
640,2025-04-07 03:00:00,46292575,Яйцеварка электрическая автоматическая E 9,271,7,2.58,8.90,62.30,0,0,0.00,0,0,18870619
641,2025-04-07 03:00:00,209076655,Яйцеварка электрическая автоматическая E 3,633,13,2.05,10.12,131.60,1,0,0.00,0,0,18870619
642,2025-04-07 03:00:00,209076655,Яйцеварка электрическая автоматическая E 3,371,6,1.62,12.98,77.89,1,0,0.00,0,0,18870619


In [27]:
# Group by 'date' (day only) and 'nmId', summing numeric columns except 'advertId'
df_grouped_guten = (
    df_guten.groupby([df_guten["date"].dt.date, "nmId", "advertId"], as_index=False)
    .agg({
        "date": "first",  # Keep the first date (to retain the day)
        "name": "first",  # Keep the first name (or customize this logic)
        "views": "sum",
        "clicks": "sum",
        "ctr": "mean",  # Sum or average, depending on your needs
        "cpc": "mean",  # Use mean for 'cpc' (cost per click)
        "sum": "sum",
        "atbs": "sum",
        "orders": "sum",
        "cr": "mean",  # Use mean for 'cr' (conversion rate)
        "shks": "sum",
        "sum_price": "sum",
        "advertId": "first"  # Keep the first 'advertId' (no summing)
    })
)

# Group by 'date' (day only) and 'nmId', summing numeric columns except 'advertId'
df_grouped_giper = (
    df_giper.groupby([df_giper["date"].dt.date, "nmId", "advertId"], as_index=False)
    .agg({
        "date": "first",  # Keep the first date (to retain the day)
        "name": "first",  # Keep the first name (or customize this logic)
        "views": "sum",
        "clicks": "sum",
        "ctr": "mean",  # Sum or average, depending on your needs
        "cpc": "mean",  # Use mean for 'cpc' (cost per click)
        "sum": "sum",
        "atbs": "sum",
        "orders": "sum",
        "cr": "mean",  # Use mean for 'cr' (conversion rate)
        "shks": "sum",
        "sum_price": "sum",
        "advertId": "first"  # Keep the first 'advertId' (no summing)
    })
)

# Group by 'date' (day only) and 'nmId', summing numeric columns except 'advertId'
df_grouped_kitchen = (
    df_kitchen.groupby([df_kitchen["date"].dt.date, "nmId", "advertId"], as_index=False)
    .agg({
        "date": "first",  # Keep the first date (to retain the day)
        "name": "first",  # Keep the first name (or customize this logic)
        "views": "sum",
        "clicks": "sum",
        "ctr": "mean",  # Sum or average, depending on your needs
        "cpc": "mean",  # Use mean for 'cpc' (cost per click)
        "sum": "sum",
        "atbs": "sum",
        "orders": "sum",
        "cr": "mean",  # Use mean for 'cr' (conversion rate)
        "shks": "sum",
        "sum_price": "sum",
        "advertId": "first"  # Keep the first 'advertId' (no summing)
    })
)

# Group by 'date' (day only) and 'nmId', summing numeric columns except 'advertId'
df_grouped_smart = (
    df_smart.groupby([df_smart["date"].dt.date, "nmId", "advertId"], as_index=False)
    .agg({
        "date": "first",  # Keep the first date (to retain the day)
        "name": "first",  # Keep the first name (or customize this logic)
        "views": "sum",
        "clicks": "sum",
        "ctr": "mean",  # Sum or average, depending on your needs
        "cpc": "mean",  # Use mean for 'cpc' (cost per click)
        "sum": "sum",
        "atbs": "sum",
        "orders": "sum",
        "cr": "mean",  # Use mean for 'cr' (conversion rate)
        "shks": "sum",
        "sum_price": "sum",
        "advertId": "first"  # Keep the first 'advertId' (no summing)
    })
)


# Rename the 'date' column to 'day' for clarity
df_grouped_guten.rename(columns={"date": "day"}, inplace=True)
df_grouped_giper.rename(columns={"date": "day"}, inplace=True)
df_grouped_kitchen.rename(columns={"date": "day"}, inplace=True)
df_grouped_smart.rename(columns={"date": "day"}, inplace=True)

df_grouped_guten['Project'] = 'WB-GutenTech'
df_grouped_giper['Project'] = 'WB-ГиперМаркет'
df_grouped_kitchen['Project'] = 'WB-KitchenAid'
df_grouped_smart['Project'] = 'WB-Smart-Market'

# Concatenate the DataFrames
df_grouped_combined_campaigns = pd.concat([df_grouped_guten, df_grouped_giper, df_grouped_kitchen, df_grouped_smart], ignore_index=True)
df_grouped_combined_campaigns['Marketplace'] = 'Wildberries'

# Display the grouped DataFrame
df_grouped_combined_campaigns

,nmId,day,name,views,clicks,ctr,cpc,sum,atbs,orders,cr,shks,sum_price,advertId,Project,Marketplace
0,25088481,2025-04-07 03:00:00,Эпилятор женский электрический Silk-epil 9-880,291,17,5.693333,4.026667,65.48,1,0,0.000000,0,0,21204285,WB-GutenTech,Wildberries
1,25623718,2025-04-07 03:00:00,Бритва мужская электрическая S3 300BT с триммером,159,9,3.713333,1.760000,18.43,1,0,0.000000,0,0,21202453,WB-GutenTech,Wildberries
2,25623718,2025-04-07 03:00:00,Бритва мужская электрическая S3 300BT с триммером,515,17,3.213333,6.466667,105.58,5,1,3.030000,1,6335,21202745,WB-GutenTech,Wildberries
3,25624340,2025-04-07 03:00:00,Электробритва Series 3 300s,284,8,2.740000,4.553333,30.33,1,0,0.000000,0,0,21202453,WB-GutenTech,Wildberries
4,25624340,2025-04-07 03:00:00,Электробритва Series 3 300s,1502,36,3.393333,11.790000,307.92,2,0,0.000000,0,0,21202745,WB-GutenTech,Wildberries
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,235599972,2025-04-07 03:00:00,Электробритва Series 9 Pro+ 9510s,529,42,7.583333,1.700000,67.41,5,0,0.000000,0,0,15562363,WB-Smart-Market,Wildberries
441,235638522,2025-04-07 03:00:00,Эпилятор Silk-epil 5 5-000,625,39,4.896667,4.743333,94.69,13,1,2.776667,1,5786,20822563,WB-Smart-Market,Wildberries
442,253299215,2025-04-07 03:00:00,Триммер для бороды и усов BT3421,555,30,4.896667,2.886667,65.43,3,1,16.666667,1,2583,19597465,WB-Smart-Market,Wildberries
443,313543155,2025-04-07 03:00:00,,1,0,0.000000,0.000000,0.05,0,0,0.000000,0,0,23236221,WB-Smart-Market,Wildberries


In [28]:
# Merge the grouped DataFrame with the filtered_df to add additional columns
df_final = df_grouped_combined_campaigns.merge(
    filtered_df[["advertId", "endTime", "createTime", "startTime","name", "status", "type"]],
    on="advertId",
    how="left"
)

# Drop the columns 'ctr', 'cpc', and 'cr'
df_final = df_final.drop(columns=["ctr", "cpc", "cr"])

# Rename the columns 'name_x' and 'name_y'
df_final.rename(
    columns={
        "name_x": "name_product",  # Rename 'name_x' to 'name_product'
        "name_y": "name_campaign"  # Rename 'name_y' to 'name_campaign'
    },
    inplace=True
)

# Display the final DataFrame
df_final
df_final_copy = df_final.copy()


In [29]:
df_final

,nmId,day,name_product,views,clicks,sum,atbs,orders,shks,sum_price,advertId,Project,Marketplace,endTime,createTime,startTime,name_campaign,status,type
0,25088481,2025-04-07 03:00:00,Эпилятор женский электрический Silk-epil 9-880,291,17,65.48,1,0,0,0,21204285,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-27,BRAUN_9 серия Эпиляторы_07.11.2024,Идут показы,Аукцион
1,25623718,2025-04-07 03:00:00,Бритва мужская электрическая S3 300BT с триммером,159,9,18.43,1,0,0,0,21202453,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания
2,25623718,2025-04-07 03:00:00,Бритва мужская электрическая S3 300BT с триммером,515,17,105.58,5,1,1,6335,21202745,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2025-01-09,BRAUN_3 серия Электробритвы_07.11.2024,Идут показы,Аукцион
3,25624340,2025-04-07 03:00:00,Электробритва Series 3 300s,284,8,30.33,1,0,0,0,21202453,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания
4,25624340,2025-04-07 03:00:00,Электробритва Series 3 300s,1502,36,307.92,2,0,0,0,21202745,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2025-01-09,BRAUN_3 серия Электробритвы_07.11.2024,Идут показы,Аукцион
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,235599972,2025-04-07 03:00:00,Электробритва Series 9 Pro+ 9510s,529,42,67.41,5,0,0,0,15562363,WB-Smart-Market,Wildberries,2100-01-01,2024-03-21,2025-02-20,Braun_Электробритвы_22.10.2024,Идут показы,Автоматическая кампания
441,235638522,2025-04-07 03:00:00,Эпилятор Silk-epil 5 5-000,625,39,94.69,13,1,1,5786,20822563,WB-Smart-Market,Wildberries,2100-01-01,2024-10-22,2025-04-21,Braun_Эпиляторы_22.10.2024,Идут показы,Автоматическая кампания
442,253299215,2025-04-07 03:00:00,Триммер для бороды и усов BT3421,555,30,65.43,3,1,1,2583,19597465,WB-Smart-Market,Wildberries,2100-01-01,2024-08-29,2025-02-20,Braun_Триммеры_22.10.2024,Идут показы,Автоматическая кампания
443,313543155,2025-04-07 03:00:00,,1,0,0.05,0,0,0,0,23236221,WB-Smart-Market,Wildberries,2100-01-01,2025-02-12,2025-03-27,Gillette_Кассеты_12.02.2025,Кампания на паузе,Автоматическая кампания


In [32]:
# Define headers for each project
headers_guten = {
    'Authorization': KeyGuten,
    'Accept': 'application/json',
    'Content-Type': 'application/json'  # Ensure this header is set
}

headers_giper = {
    'Authorization': KeyGiper,
    'Accept': 'application/json',
    'Content-Type': 'application/json'  # Ensure this header is set
}

headers_kitchen = {
    'Authorization': KeyKitchen,
    'Accept': 'application/json',
    'Content-Type': 'application/json'  # Ensure this header is set
}

headers_smart = {
    'Authorization': KeySmart,
    'Accept': 'application/json',
    'Content-Type': 'application/json'  # Ensure this header is set

}

# Define the API endpoint
url = "https://seller-analytics-api.wildberries.ru/api/v2/nm-report/detail"

# Set fixed date range for testing
yesterday_start = '2025-04-07 00:00:00'
yesterday_end = '2025-04-07 23:59:59'

def get_report(url, headers, begin, end, page, project_name):
    all_data = []
    max_retries = 3
    retry_delay = 30  # seconds
    request_timeout = 60  # seconds
    
    while True:
        for attempt in range(max_retries):
            try:
                # Define the request body with all required parameters
                request_body = {
                    "period": {
                        "begin": begin,
                        "end": end
                    },
                    "orderBy": {
                        "field": "ordersSumRub",
                        "mode": "desc"
                    },
                    "page": page,
                    "timezone": "Europe/Moscow",  # Added as recommended in API docs
                    "brandNames": [],  # Empty array to get all brands
                    "objectIDs": [],   # Empty array to get all objects
                    "nmIDs": []        # Empty array to get all items
                }

                json_data = json.dumps(request_body, ensure_ascii=False)
                
                # Send the POST request with timeout
                response = requests.post(
                    url, 
                    headers=headers, 
                    data=json_data, 
                    timeout=request_timeout
                )
                
                # Handle rate limiting (429 status code)
                if response.status_code == 429:
                    wait_time = int(response.headers.get('Retry-After', retry_delay))
                    print(f"Rate limit exceeded for {project_name}. Waiting {wait_time} seconds...")
                    time.sleep(wait_time)
                    continue
                
                # Check for successful response
                if response.status_code != 200:
                    print(f"Request failed with status code {response.status_code} for {project_name}, page {page}")
                    print("Response text:", response.text)
                    if attempt == max_retries - 1:
                        return all_data
                    time.sleep(retry_delay)
                    continue
                
                data = response.json()
                
                # Validate response structure
                if not data.get('data') or not isinstance(data['data'].get('cards'), list):
                    print(f"Invalid data structure received for {project_name}, page {page}")
                    return all_data
                
                cards = data['data']['cards']
                
                # Check if we have any data
                if not cards:
                    print(f"No more data available for {project_name}")
                    return all_data
                
                all_data.extend(cards)
                print(f"Page {page} retrieved successfully for {project_name} (got {len(cards)} items)")
                
                # Check if there are more pages (using isNextPage flag)
                if not data['data'].get('isNextPage', False):
                    print(f"Reached last page for {project_name}")
                    return all_data
                
                # Prepare for next page
                page += 1
                time.sleep(5)  # Reduced delay between pages to 5 seconds
                break  # Success, break out of retry loop
                
            except requests.exceptions.RequestException as e:
                print(f"Request failed for {project_name}, page {page}: {str(e)}")
                if attempt == max_retries - 1:
                    return all_data
                time.sleep(retry_delay)
                continue
            except Exception as e:
                print(f"Unexpected error for {project_name}, page {page}: {str(e)}")
                return all_data

# Get data for each project
data_guten = get_report(url, headers_guten, yesterday_start, yesterday_end, 1, 'WB-GutenTech')
data_giper = get_report(url, headers_giper, yesterday_start, yesterday_end, 1, 'WB-ГиперМаркет')
data_kitchen = get_report(url, headers_kitchen, yesterday_start, yesterday_end, 1, 'WB-KitchenAid')
data_smart = get_report(url, headers_smart, yesterday_start, yesterday_end, 1, 'WB-Smart-Market')


# Function to flatten the JSON data for the current period
def flatten_json_current_period(cards):
    flattened_data = []
    for card in cards:
        nmID = card["nmID"]
        vendorCode = card["vendorCode"]
        brandName = card["brandName"]
        objectID = card["object"]["id"]
        objectName = card["object"]["name"]
        

        # Extract statistics for the selected period
        selected_period = card["statistics"]["selectedPeriod"]

        flattened_data.append({
            "nmID": nmID,
            "vendorCode": vendorCode,
            "brandName": brandName,
            "objectID": objectID,
            "objectName": objectName,
            "begin": selected_period["begin"],
            "end": selected_period["end"],
            "openCardCount": selected_period["openCardCount"],
            "addToCartCount": selected_period["addToCartCount"],
            "ordersCount": selected_period["ordersCount"],
            "ordersSumRub": selected_period["ordersSumRub"],
            "buyoutsCount": selected_period["buyoutsCount"],
            "buyoutsSumRub": selected_period["buyoutsSumRub"],
            "cancelCount": selected_period["cancelCount"],
            "cancelSumRub": selected_period["cancelSumRub"],
            "avgOrdersCountPerDay": selected_period["avgOrdersCountPerDay"],
            "avgPriceRub": selected_period["avgPriceRub"],
            "addToCartPercent": selected_period["conversions"]["addToCartPercent"],
            "cartToOrderPercent": selected_period["conversions"]["cartToOrderPercent"],
            "buyoutsPercent": selected_period["conversions"]["buyoutsPercent"],
            "stocksMp": card["stocks"]["stocksMp"],
            "stocksWb": card["stocks"]["stocksWb"]
        })

    return flattened_data

# Convert the flattened data to a DataFrame
flattened_data_guten = flatten_json_current_period(data_guten)
flattened_data_giper = flatten_json_current_period(data_giper)
flattened_data_kitchen = flatten_json_current_period(data_kitchen)
flattened_data_smart = flatten_json_current_period(data_smart)

df_guten = pd.DataFrame(flattened_data_guten)
df_giper = pd.DataFrame(flattened_data_giper)
df_kitchen = pd.DataFrame(flattened_data_kitchen)
df_smart = pd.DataFrame(flattened_data_smart)

# Add the 'Project' column to each DataFrame before concatenation
df_guten['Project'] = 'WB-GutenTech'
df_giper['Project'] = 'WB-ГиперМаркет'
df_kitchen['Project'] = 'WB-KitchenAid'
df_smart['Project'] = 'WB-Smart-Market'

# Combine all campaign data
combined_df = pd.concat([df_guten, df_giper, df_kitchen, df_smart], ignore_index=True)
combined_df['Marketplace'] = 'Wildberries'
print("Columns in combined_campaigns:", combined_df.columns.tolist())

# Display the DataFrame
combined_df


# Display the combined DataFrame
print("Combined DataFrame:\n", combined_df.head())
df_copy=combined_df.copy()

Page 1 retrieved successfully for WB-GutenTech (got 1000 items)
Page 2 retrieved successfully for WB-GutenTech (got 1000 items)
Page 3 retrieved successfully for WB-GutenTech (got 1000 items)
Rate limit exceeded for WB-GutenTech. Waiting 30 seconds...
Page 4 retrieved successfully for WB-GutenTech (got 1000 items)
Page 5 retrieved successfully for WB-GutenTech (got 1000 items)
Rate limit exceeded for WB-GutenTech. Waiting 30 seconds...
Page 6 retrieved successfully for WB-GutenTech (got 1000 items)
Page 7 retrieved successfully for WB-GutenTech (got 1000 items)
Page 8 retrieved successfully for WB-GutenTech (got 1000 items)
Rate limit exceeded for WB-GutenTech. Waiting 30 seconds...
Page 9 retrieved successfully for WB-GutenTech (got 1000 items)
Page 10 retrieved successfully for WB-GutenTech (got 814 items)
Reached last page for WB-GutenTech
Page 1 retrieved successfully for WB-ГиперМаркет (got 1000 items)
Page 2 retrieved successfully for WB-ГиперМаркет (got 179 items)
Reached last p

In [33]:
df_copy=combined_df.copy()
df_final_copy=df_final.copy()

In [34]:
df_copy

,nmID,vendorCode,brandName,objectID,objectName,begin,end,openCardCount,addToCartCount,ordersCount,...,cancelSumRub,avgOrdersCountPerDay,avgPriceRub,addToCartPercent,cartToOrderPercent,buyoutsPercent,stocksMp,stocksWb,Project,Marketplace
0,159121857,I01003,AND,636,Тонометры,2025-04-07 00:00:00,2025-04-07 23:59:59,306,44,15,...,6078,15,3040,14,34,86,0,186,WB-GutenTech,Wildberries
1,159121847,I01002,AND,636,Тонометры,2025-04-07 00:00:00,2025-04-07 23:59:59,519,63,19,...,2378,19,2378,12,30,94,0,119,WB-GutenTech,Wildberries
2,159488075,I01475,AND,594,Массажеры электрические,2025-04-07 00:00:00,2025-04-07 23:59:59,491,40,10,...,0,10,3998,8,25,100,0,308,WB-GutenTech,Wildberries
3,252275489,841332,ELIKOR,2184,Вытяжки кухонные,2025-04-07 00:00:00,2025-04-07 23:59:59,336,32,6,...,11980,6,5987,10,19,60,1810,63,WB-GutenTech,Wildberries
4,236561143,7500435225069,Braun,634,Эпиляторы,2025-04-07 00:00:00,2025-04-07 23:59:59,85,7,4,...,7571,4,7571,8,57,75,0,8,WB-GutenTech,Wildberries
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11231,288465483,8700216589079,GILLETTE,1527,Бритвы безопасные,2025-04-07 00:00:00,2025-04-07 23:59:59,6,0,0,...,0,0,0,0,0,0,0,84,WB-Smart-Market,Wildberries
11232,288465484,8700216589017,GILLETTE,1527,Бритвы безопасные,2025-04-07 00:00:00,2025-04-07 23:59:59,8,1,0,...,0,0,0,13,0,0,0,61,WB-Smart-Market,Wildberries
11233,288465485,8700216588713,GILLETTE,1527,Бритвы безопасные,2025-04-07 00:00:00,2025-04-07 23:59:59,5,0,0,...,0,0,0,0,0,0,0,5,WB-Smart-Market,Wildberries
11234,288465487,8700216588911,GILLETTE,1527,Бритвы безопасные,2025-04-07 00:00:00,2025-04-07 23:59:59,9,0,0,...,0,0,0,0,0,0,0,73,WB-Smart-Market,Wildberries


In [35]:
df_unique = df_copy['nmID'].unique()
print(df_unique)

[159121857 159121847 159488075 ... 288465485 288465487 313543156]


In [36]:
# Assuming your DataFrame is named df
specific_nmID = 137609342

# Filter the DataFrame for the specific nmID
filtered_df_id = df_copy[df_copy['nmID'] == specific_nmID]

# Group by nmID and sum the ordersSumRub
total_ordersSumRub = filtered_df_id['ordersSumRub'].sum()

print(f"Total ordersSumRub for nmID {specific_nmID}: {total_ordersSumRub}")

Total ordersSumRub for nmID 137609342: 0


In [37]:
df_unique = df_copy['ordersSumRub'].sum()
print(df_unique)

1884420


In [39]:
# Convert 'day' and 'dt' to datetime for accurate merging
df_final_copy['day'] = pd.to_datetime(df_final_copy['day']).dt.date
df_copy['begin'] = pd.to_datetime(df_copy['begin']).dt.date

# Rename columns in df2 to match df1 for merging
df_copy.rename(columns={'nmID': 'nmId', 'begin': 'day'}, inplace=True)

# Merge the DataFrames on 'nmId', 'day', and 'Project'
merged_df_2 = pd.merge(
    df_final_copy,
    df_copy[['nmId', 'day', 'ordersCount', 'ordersSumRub','addToCartCount']],
    on=['nmId', 'day'],
    how='left'
)

# Fill NaN values with 0 for ordersCount and ordersSumRub
merged_df_2['ordersCount'].fillna(0, inplace=True)
merged_df_2['ordersSumRub'].fillna(0, inplace=True)
merged_df_2['addToCartCount'].fillna(0, inplace=True)


# Display the merged DataFrame
merged_df_2

C:\Users\User\AppData\Local\Temp\ipykernel_5148\2097599021.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df_2['ordersCount'].fillna(0, inplace=True)
C:\Users\User\AppData\Local\Temp\ipykernel_5148\2097599021.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exa

,nmId,day,name_product,views,clicks,sum,atbs,orders,shks,sum_price,...,Marketplace,endTime,createTime,startTime,name_campaign,status,type,ordersCount,ordersSumRub,addToCartCount
0,25088481,2025-04-07,Эпилятор женский электрический Silk-epil 9-880,291,17,65.48,1,0,0,0,...,Wildberries,2100-01-01,2024-11-07,2024-11-27,BRAUN_9 серия Эпиляторы_07.11.2024,Идут показы,Аукцион,0,0,1
1,25623718,2025-04-07,Бритва мужская электрическая S3 300BT с триммером,159,9,18.43,1,0,0,0,...,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания,1,8020,6
2,25623718,2025-04-07,Бритва мужская электрическая S3 300BT с триммером,515,17,105.58,5,1,1,6335,...,Wildberries,2100-01-01,2024-11-07,2025-01-09,BRAUN_3 серия Электробритвы_07.11.2024,Идут показы,Аукцион,1,8020,6
3,25624340,2025-04-07,Электробритва Series 3 300s,284,8,30.33,1,0,0,0,...,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания,0,0,4
4,25624340,2025-04-07,Электробритва Series 3 300s,1502,36,307.92,2,0,0,0,...,Wildberries,2100-01-01,2024-11-07,2025-01-09,BRAUN_3 серия Электробритвы_07.11.2024,Идут показы,Аукцион,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,235599972,2025-04-07,Электробритва Series 9 Pro+ 9510s,529,42,67.41,5,0,0,0,...,Wildberries,2100-01-01,2024-03-21,2025-02-20,Braun_Электробритвы_22.10.2024,Идут показы,Автоматическая кампания,0,0,6
441,235638522,2025-04-07,Эпилятор Silk-epil 5 5-000,625,39,94.69,13,1,1,5786,...,Wildberries,2100-01-01,2024-10-22,2025-04-21,Braun_Эпиляторы_22.10.2024,Идут показы,Автоматическая кампания,3,18897,21
442,253299215,2025-04-07,Триммер для бороды и усов BT3421,555,30,65.43,3,1,1,2583,...,Wildberries,2100-01-01,2024-08-29,2025-02-20,Braun_Триммеры_22.10.2024,Идут показы,Автоматическая кампания,3,10329,5
443,313543155,2025-04-07,,1,0,0.05,0,0,0,0,...,Wildberries,2100-01-01,2025-02-12,2025-03-27,Gillette_Кассеты_12.02.2025,Кампания на паузе,Автоматическая кампания,2,9592,3


In [40]:
df_unique = merged_df_2['ordersSumRub'].sum()
print(df_unique)


1440109


## Insert the data

In [41]:
password = os.getenv('ClickHouse')
# Define connection parameters
client = get_client(
    host='rc1a-j5ou9lq30ldal602.mdb.yandexcloud.net',  # Your Yandex Cloud ClickHouse host
    port=8443,                                          # Yandex Cloud uses port 8443 for HTTPS
    username='user1',                                      # Your ClickHouse username
    password= password,                           # Your ClickHouse password
    database='user1',                            # Your database name
    secure=True,                                        # Use HTTPS
    verify=False                                        # Disable SSL certificate verification 
    # Define the data to insert
)

In [43]:
# Ensure date columns are in the correct format for ClickHouse
merged_df_2['day'] = pd.to_datetime(merged_df_2['day'])  # Convert to datetime


# Debugging: Check the data types of the DataFrame
print("Data types of merged_df:")
print(merged_df_2.dtypes)

# Ensure the DataFrame has the correct columns
columns = [
    'nmId', 'day', 'name_product', 'views', 'clicks', 'sum', 'atbs', 'orders', 'shks',
    'sum_price', 'advertId', 'Project', 'Marketplace','endTime', 'createTime', 'startTime', 
    'name_campaign', 'status', 'type','ordersCount', 'ordersSumRub','addToCartCount'
]

# Reorder columns to match the expected order
merget_df_copy_2 = merged_df_2[columns]

# Convert DataFrame to a list of tuples for bulk insertion
data = [tuple(row) for row in merget_df_copy_2.to_numpy()]

# Debugging: Check the structure of the data
print("Sample data to insert:", data[:5])  # Print the first 5 rows to check the structure

# Define the table name
table_name = 'campaign_data_wb'

# Use the insert method for bulk insertion
client.insert(table_name, data, column_names=columns)
print("Data inserted successfully!")

Data types of merged_df:
nmId                       int64
day               datetime64[ns]
name_product              object
views                      int64
clicks                     int64
sum                      float64
atbs                       int64
orders                     int64
shks                       int64
sum_price                  int64
advertId                   int64
Project                   object
Marketplace               object
endTime                   object
createTime                object
startTime                 object
name_campaign             object
status                    object
type                      object
ordersCount                int64
ordersSumRub               int64
addToCartCount             int64
dtype: object
Sample data to insert: [(25088481, Timestamp('2025-04-07 00:00:00'), 'Эпилятор женский электрический Silk-epil 9-880', 291, 17, 65.48, 1, 0, 0, 0, 21204285, 'WB-GutenTech', 'Wildberries', datetime.date(2100, 1, 1), datetime.date(2024, 